In [ ]:
import pandas as pd

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../hourly-egrid/")
from src.column_checks import get_dtypes
import src.load_data as load_data

# NOX

Control-specific emission rates:
- [x] Load control-specific nox rates from EIA-923 8C
- [x] Load control-boiler associations
- [ ] Load control attributes
- [ ] Map controls to boilers
- [ ] Map boilers to generators 
- [ ] Calculate generator-specific emission rates

Boiler firing types

In [ ]:
year = 2020
eia_filepath = "../../../EIA Data/"

emissions_controls_eia923_names = [
    "report_date",
    "plant_id_eia",
    "equipment_tech_description",
    "pm_control_id",
    "so2_control_id",
    "nox_control_id",
    "mercury_control_id",
    "operational_status",
    "hours_in_service",
    "annual_nox_emission_rate_lb_per_mmbtu",
    "ozone_season_nox_emission_rate_lb_per_mmbtu",
    "pm_emission_rate_lb_per_mmbtu",
    "pm_removal_efficiency_annual",
    "pm_removal_efficiency_at_full_load",
    "pm_test_date",
    "so2_removal_efficiency_annual",
    "so2_removal_efficiency_at_full_load",
    "so2_test_date",
    "fgd_sorbent_consumption_1000_tons",
    "fgd_electricity_consumption_mwh",
    "hg_removal_efficiency",
    "hg_emission_rate_lb_per_trillion_btu",
    "acid_gas_removal_efficiency",
]

emissions_controls_eia923 = pd.read_excel(
    io=(
        eia_filepath
        + f"f923_{year}/EIA923_Schedule_8_Annual_Environmental_Information_{year}_Final_Revision.xlsx"
    ),
    sheet_name="8C Air Emissions Control Info",
    header=4,
    names=emissions_controls_eia923_names,
    dtype=get_dtypes(),
    na_values=".",
    parse_dates=["report_date","pm_test_date","so2_test_date"]
)

emissions_controls_eia923

In [ ]:
boiler_nox_association_eia860_names = [
    "utility_id_eia",
    "utility_name_eia",
    "plant_id_eia",
    "plant_name_eia",
    "boiler_id",
    "nox_control_id",
    "steam_plant_type",
]

boiler_nox_association_eia860 = pd.read_excel(
    io=(
        eia_filepath
        + f"eia860{year}/6_1_EnviroAssoc_Y{year}.xlsx"
    ),
    sheet_name="Boiler NOx",
    header=1,
    names=boiler_nox_association_eia860_names,
    dtype=get_dtypes(),
    na_values=".",
    skipfooter=1
)

boiler_nox_association_eia860

In [ ]:
pudl_out = load_data.initialize_pudl_out(year)

In [ ]:
bga = pudl_out.bga_eia860()

### Combine data

In [ ]:
nox_rates = emissions_controls_eia923[["plant_id_eia","nox_control_id","annual_nox_emission_rate_lb_per_mmbtu","ozone_season_nox_emission_rate_lb_per_mmbtu",]].dropna(thresh=2)
nox_rates

In [ ]:
boiler_nox_rate = boiler_nox_association_eia860[["plant_id_eia","boiler_id","nox_control_id"]]
boiler_nox_rate.merge(nox_rates, how="left", on=["plant_id_eia","nox_control_id"]).head(20)